# TPM - Atividade II

Crie uma tabela com os dados de Retorno, Volatilidade, Alfa, Beta, Sharpe e Sortino para cada um dos portfólios.

In [41]:
import pandas as pd
import numpy as np

import config as cfg
import helpers as hlp

In [42]:
# Lendo a lista de portfólios
df_portfolio_list = pd.read_csv(f'{cfg.PATH_RESULTADOS}/portfolio_files.csv')

df_portfolio_list.head()

,Portfolio,File Name
0,Blue Chips,blue_chips.parquet
1,Mid Caps,mid_caps.parquet
2,Small Caps,small_caps.parquet
3,Micro Caps,micro_caps.parquet
4,High Return,high_return.parquet


## 1. Calculando as variáveis para o portfólio Blue Chips

In [43]:
# Recuperando o arquivo
portfolio_name = df_portfolio_list['Portfolio'].iloc[0]
file_name = df_portfolio_list['File Name'].iloc[0]

df_blue_chips = pd.read_parquet(f"{cfg.PATH_CALCULATED_PORTFOLIO_DIR}/{file_name}", columns=['TOTAL AMOUNT']).rename(columns={'TOTAL AMOUNT': portfolio_name})

df_blue_chips.head()

,Blue Chips
Data,
2019-12-30,10000010.79
2020-01-02,10268945.33
2020-01-03,10229811.19
2020-01-06,10116699.86
2020-01-07,10155441.23


In [44]:
def log_return(df, column):
    return np.log(df[column]/df[column].shift(1))

In [45]:
LOG_COLUMN_NAME = "Log Return"

df_blue_chips[LOG_COLUMN_NAME] = log_return(df_blue_chips, portfolio_name)

In [46]:
# Retorno acumulado do fundo
blue_chips_return = df_blue_chips[LOG_COLUMN_NAME].sum()

# A volatilidade 
blue_chips_vol = df_blue_chips[LOG_COLUMN_NAME].std()

In [48]:
print(f'Log Return Acum => {blue_chips_return * 100}')
print(f'Volatility => {blue_chips_vol * 100}')

Log Return Acum => 14.187889007710336
Volatility => 1.8249966967893265


In [47]:
# Buscando os dados da IBOV e do DI do período
IBOV = '.BVSP'
DI_INDEX = 'DI_INDEX'

df_prelast = pd.read_parquet(cfg.PATH_PRELAST_PARQUET, columns=['Data', IBOV, DI_INDEX])
df_prelast.set_index('Data', inplace=True)

In [49]:
# Calulando os retornos
IBOV_RETURN = f'{IBOV} Log Return'
DI_INDEX_RETURN = f'{DI_INDEX} Log Return'

df_prelast[IBOV_RETURN] = log_return(df_prelast, IBOV)
df_prelast[DI_INDEX_RETURN] = log_return(df_prelast, DI_INDEX)

bvsp_return = df_prelast[IBOV_RETURN].sum()
di_return = df_prelast[DI_INDEX_RETURN].sum()

print(f'BVSP Log Return Acum => {bvsp_return * 100}')
print(f'DI_INDEX Log Return Acum => {di_return * 100}')

BVSP Log Return Acum => -12.671084996483192
DI_INDEX Log Return Acum => 21.9126697944389


In [50]:
# Gerando data frame para cálculo do beta
base_ativos = pd.merge(df_prelast[IBOV_RETURN], df_blue_chips[LOG_COLUMN_NAME], left_index=True, right_index=True)
base_ativos.columns = [IBOV, portfolio_name]
base_ativos.dropna(inplace=True)

base_ativos.head()

,.BVSP,Blue Chips
Data,,
2020-01-02,0.025002,0.026538
2020-01-03,-0.007334,-0.003818
2020-01-06,-0.007066,-0.011119
2020-01-07,-0.001850,0.003822
2020-01-08,-0.003563,-0.002072


In [51]:
# Calcule a matriz de covariância para o mercado e o ativo
#cov_matrix = np.cov(base_ativos.iloc[:,0], base_ativos.iloc[:,1])
cov_matrix = np.cov(base_ativos[IBOV], base_ativos[portfolio_name])

# Extraia a covariância entre o mercado e o ativo
market_asset_cov = cov_matrix[0,1]

# Calcule o beta do ativo (ou seja, a sensibilidade dos retornos do ativo aos retornos do mercado)
beta = market_asset_cov / np.var(base_ativos[IBOV])

In [52]:
expected_return = di_return + beta * (bvsp_return - di_return)

alfa = blue_chips_return - expected_return

In [53]:
# Calculando indice sharp
sharpe_index = (blue_chips_return - di_return)/blue_chips_vol

# Sortino
avg_return = df_blue_chips[LOG_COLUMN_NAME].mean()
downside_deviation = np.sqrt(np.mean(np.square(np.minimum(df_blue_chips[LOG_COLUMN_NAME] - avg_return, 0))))
sortino_index = (blue_chips_return - di_return)/downside_deviation

In [54]:
print(f'Log Return Acum => {blue_chips_return * 100}')
print(f'Volatility => {blue_chips_vol * 100}')
print(f'expected return => {expected_return * 100}')
print(f'alfa => {alfa * 100}')
print(f'beta => {beta}')
print(f'sharpe index => {sharpe_index}')
print(f'sortino index => {sortino_index}')

Log Return Acum => 14.187889007710336
Volatility => 1.8249966967893265
expected return => -10.12242350066245
alfa => 24.310312508372785
beta => 0.9263046620811185
sharpe index => -4.232764256679801
sortino index => -5.539002044226097


## 2. Calculando para todos os fundos e gerando a tabela sumarizada

In [55]:
def calculate_log_return(df, column):
    return np.log(df[column]/df[column].shift(1))

def calculate_beta(df, market_name, asset_name):
    # Calcule a matriz de covariância para o mercado e o ativo
    cov_matrix = np.cov(df[market_name], df[asset_name])

    # Extraia a covariância entre o mercado e o ativo
    market_asset_cov = cov_matrix[0,1]

    # Calcule o beta do ativo (ou seja, a sensibilidade dos retornos do ativo aos retornos do mercado)
    return market_asset_cov / np.var(df[market_name])

def calculate_expected_return(free_risk_return, beta, benchmark_return):
    return free_risk_return + beta * (benchmark_return - free_risk_return)

def calculate_alfa(asset_return, expected_return):
    return asset_return - expected_return

def calculate_downside_deviation(df, asset_name):
    avg_return = df[asset_name].mean()
    return np.sqrt(np.mean(np.square(np.minimum(df[asset_name] - avg_return, 0))))

# As formulas dos índices Sharp e Sortino são iguais só mudando o desvio (um usa a vol o outro desvio negativo)
def caculate_sharpe_sortino_index(asset_return, free_risk_return, deviation):
    return (asset_return - free_risk_return) / deviation


In [56]:
# ------------------------------------------------------------------------------------------------
# Gerando inicializando um data fram dos retornos com os retornos do IBOV e do DI
# ------------------------------------------------------------------------------------------------
IBOV = '.BVSP'
DI_INDEX = 'DI_INDEX'

# Buscando os dados dos dois índices
df_prelast = pd.read_parquet(cfg.PATH_PRELAST_PARQUET, columns=['Data', IBOV, DI_INDEX])
df_prelast.set_index('Data', inplace=True)

df_returns = pd.DataFrame()
df_returns[IBOV] = calculate_log_return(df_prelast, IBOV)
df_returns[DI_INDEX] = calculate_log_return(df_prelast, DI_INDEX)

df_returns.head()

,.BVSP,DI_INDEX
Data,,
2019-12-30,NaN,NaN
2020-01-02,0.025002,0.000342
2020-01-03,-0.007334,0.000171
2020-01-06,-0.007066,0.000171
2020-01-07,-0.001850,0.000171


In [57]:
#Acrescentando o retorno dos portfólios
for index, row in df_portfolio_list.iterrows():
    portfolio_name = row['Portfolio']
    file_name = row['File Name']

    df = pd.read_parquet(f"{cfg.PATH_CALCULATED_PORTFOLIO_DIR}/{file_name}", columns=['TOTAL AMOUNT']).rename(columns={'TOTAL AMOUNT': portfolio_name})
    
    df_returns[portfolio_name] = calculate_log_return(df, portfolio_name)

df_returns.head()

,.BVSP,DI_INDEX,Blue Chips,Mid Caps,Small Caps,Micro Caps,High Return,Low Return,High Volatility,Low Volatility,...,Low Alpha,High Sharpe,Low Sharpe,High P/E,Low P/E,High P/B,Low P/B,High Debt,Low Debt,Banks
Data,,,,,,,,,,,,,,,,,,,,,
2019-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-02,0.025002,0.000342,0.026538,0.009043,0.002578,0.018680,0.013905,0.016532,0.016652,0.000343,...,0.018643,0.017151,0.019112,0.006910,0.011588,0.019112,0.006553,0.018512,0.003678,0.015512
2020-01-03,-0.007334,0.000171,-0.003818,0.007180,0.003785,0.013293,0.014767,0.011288,0.005648,0.002840,...,0.002900,0.016435,-0.001191,-0.001441,0.003807,0.011790,0.007781,0.021835,0.005909,0.014771
2020-01-06,-0.007066,0.000171,-0.011119,-0.001473,-0.004627,0.014201,-0.010367,0.004893,0.000739,-0.002813,...,0.007045,0.005094,-0.000414,0.000776,-0.001999,0.003061,0.004478,0.003610,0.008522,-0.009485
2020-01-07,-0.001850,0.000171,0.003822,0.008257,0.012669,0.030619,0.001029,-0.002905,0.005934,0.012414,...,-0.013465,-0.006649,-0.006763,0.004683,0.007665,0.013982,-0.001947,0.009050,0.030240,0.007489


In [58]:
# Eliminando o N/A
df_returns.dropna(inplace = True)

# Garantindo que os retornos estão ordenados
df_returns.sort_index(inplace = True)

# Mantendo a data base em 30/03/2023
df_returns.drop(index=['2023-03-31'], inplace = True)

In [59]:
bvsp_return = df_returns[IBOV].sum()
di_return = df_returns[DI_INDEX].sum()
data_referencia = df_returns.index.max()

portfolio_indexes_list = [{
                'Portfolio' : IBOV, 
                'Retorno' : bvsp_return * 100,
                'Volatilidade' : df_returns[IBOV].std() * 100, 
                'Retorno Experado' : np.nan,
                'Alfa' : np.nan,
                'Beta' : np.nan,
                'Sharpe' : np.nan,
                'Sortino' : np.nan,                
                'Data Ref.' : data_referencia
            },
            {
                'Portfolio' : DI_INDEX, 
                'Retorno' : di_return * 100,
                'Volatilidade' : np.nan, 
                'Retorno Experado' : np.nan,
                'Alfa' : np.nan,
                'Beta' : np.nan,
                'Sharpe' : np.nan,
                'Sortino' : np.nan,                
                'Data Ref.' : data_referencia
            }
            ]

for index, row in df_portfolio_list.iterrows():
    portfolio_name = row['Portfolio']

    # Calculando indicadores do Portfolio
    acum_return = df_returns[portfolio_name].sum()
    vol_portfolio = df_returns[portfolio_name].std()
    downside_deviation = calculate_downside_deviation(df_returns, portfolio_name)
    beta = calculate_beta(df_returns, IBOV, portfolio_name)
    expected_return = calculate_expected_return(di_return, beta, bvsp_return)
    alfa = calculate_alfa(acum_return,expected_return)
    sharpe_index = caculate_sharpe_sortino_index(acum_return, di_return, vol_portfolio)
    sortino_index = caculate_sharpe_sortino_index(acum_return, di_return, downside_deviation)

    row = {
            'Portfolio' : portfolio_name, 
            'Retorno' : acum_return * 100,
            'Volatilidade' : vol_portfolio * 100, 
            'Retorno Experado' : expected_return * 100,
            'Alfa' : alfa * 100,
            'Beta' : beta,
            'Sharpe' : sharpe_index,
            'Sortino' : sortino_index,
            'Data Ref.' : data_referencia
        }

    portfolio_indexes_list.append(row)

df_portfolio_index = pd.DataFrame(portfolio_indexes_list)
df_portfolio_index.head()

,Portfolio,Retorno,Volatilidade,Retorno Experado,Alfa,Beta,Sharpe,Sortino,Data Ref.
0,.BVSP,-10.889629,1.925939,NaN,NaN,NaN,NaN,NaN,2023-03-30
1,DI_INDEX,21.861894,NaN,NaN,NaN,NaN,NaN,NaN,2023-03-30
2,Blue Chips,16.025721,1.824957,-8.471746,24.497467,0.926175,-3.197978,-4.183808,2023-03-30
3,Mid Caps,10.626336,1.780594,-6.073937,16.700274,0.852963,-6.310006,-8.137238,2023-03-30
4,Small Caps,23.793951,1.568962,-0.689106,24.483057,0.688548,1.231423,1.549938,2023-03-30


In [60]:
# Salvando os dados
hlp.save_to_parquet(df_returns, cfg.PATH_TABELA_RETORNOS)
hlp.save_to_parquet(df_portfolio_index, cfg.PATH_TABELA_INDICADORES_PORTFOLIO)